# 综合案例：带分流的运河分层控制

本案例实现了一个用户描述的仿真场景：一个带有水闸和分流出口的运河系统，并受到多种扰动。其控制架构是分层的，由一个中央调度器（MPC/基于规则）为本地PID控制器设定目标。

## 系统拓扑

上游来水 -> **[上游运河]** -> **[分流口]** -> **[控制水闸]** -> **[下游运河]**

## 关键组件

- **物理组件**: 两段运河，一个水闸。
- **智能体 (Agents)**:
  - `DigitalTwinAgent`: 用于上游运河的数字孪生。
  - `LocalControlAgent (PID)`: 用于水闸的本地PID控制器。
  - `CentralDispatcher`: 用于高级设定点控制的中央调度器。
  - `ARIMAForecaster`: 用于预测上游来水。
  - `RainfallAgent`: 模拟风暴扰动。
  - `WaterUseAgent`: 模拟分流口的流出。

仿真将经历正常、扰动和恢复三个阶段，以测试控制策略的韧性和有效性。

### 1. 导入必要的库

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 导入所有需要的平台组件
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.canal import Canal
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.io.physical_io_agent import PhysicalIOAgent
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher
from swp.local_agents.prediction.arima_forecaster import ARIMAForecaster
from swp.simulation_identification.disturbances.rainfall_agent import RainfallAgent
from swp.simulation_identification.disturbances.water_use_agent import WaterUseAgent

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号

### 2. 加载配置文件

我们将所有的参数都存储在一个JSON文件中，使仿真逻辑与具体参数解耦。

In [ ]:
CONFIG_PATH = 'examples/canal_diversion_simulation.json'

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置文件加载成功！")

### 3. 定义仿真运行函数

这个函数将根据加载的配置来搭建和运行整个仿真系统。

In [ ]:
def run_simulation_from_config(cfg):
    print("--- 开始搭建运河分流仿真系统 ---")

    # 提取配置
    sim_cfg = cfg['simulation_settings']
    topics = cfg['messaging_topics']
    comp_cfgs = cfg['components']
    agent_cfgs = cfg['agents']

    message_bus = MessageBus()

    # --- 配置物理组件 ---
    uc_cfg = comp_cfgs['upstream_canal']
    canal_length = uc_cfg['params']['length']
    canal_bottom_width = uc_cfg['params']['bottom_width']
    canal_side_slope = uc_cfg['params']['side_slope_z']
    initial_water_level = uc_cfg['initial_water_level']
    initial_volume = canal_length * (canal_bottom_width * initial_water_level + canal_side_slope * initial_water_level**2)

    upstream_canal = Canal(
        name="upstream_canal",
        initial_state={'volume': initial_volume, 'water_level': initial_water_level, 'outflow': sim_cfg['normal_inflow']},
        parameters=uc_cfg['params'],
        message_bus=message_bus,
        inflow_topic=topics['upstream_inflow_data']
    )

    cg_cfg = comp_cfgs['control_gate']
    control_gate = Gate(name="control_gate", initial_state=cg_cfg['initial_state'], parameters=cg_cfg['params'])

    dc_cfg = comp_cfgs['downstream_canal']
    downstream_canal = Canal(
        name="downstream_canal",
        initial_state={'volume': initial_volume * dc_cfg['initial_state']['volume_ratio'], 'water_level': initial_water_level * dc_cfg['initial_state']['level_ratio'], 'outflow': dc_cfg['initial_state']['outflow']},
        parameters=dc_cfg['params']
    )
    physical_components = [upstream_canal, control_gate, downstream_canal]
    print("物理组件配置完成。")

    # --- 配置智能体组件 ---
    agent_components = []

    # 数字孪生
    twin_agent_upstream = DigitalTwinAgent("twin_agent_upstream", upstream_canal, message_bus, topics['upstream_state'])
    agent_components.append(twin_agent_upstream)

    # PID控制器
    pid_cfg = agent_cfgs['gate_control_agent']['pid_controller']
    pid_controller = PIDController(setpoint=pid_cfg['setpoint'], Kp=pid_cfg['Kp'], Ki=pid_cfg['Ki'], Kd=pid_cfg['Kd'], min_output=0.0, max_output=cg_cfg['params']['max_opening'])
    gate_control_agent = LocalControlAgent("gate_control_agent", pid_controller, message_bus, topics['upstream_state'], 'water_level', topics['gate_action'], sim_cfg['dt'], topics['gate_control_setpoint'])
    agent_components.append(gate_control_agent)

    # 中央调度器
    disp_cfg = agent_cfgs['central_dispatcher']
    central_dispatcher = CentralDispatcher("central_dispatcher", message_bus, {'upstream_level': topics['upstream_state']}, {'inflow_forecast': topics['upstream_inflow_prediction']}, {'gate_setpoint': topics['gate_control_setpoint']}, disp_cfg['rules'])
    agent_components.append(central_dispatcher)

    # 预测器
    fc_cfg = agent_cfgs['rainfall_forecaster']['config']
    forecaster_config = {**fc_cfg, 'observation_topic': topics['upstream_state'], 'forecast_topic': topics['upstream_inflow_prediction']}
    rainfall_forecaster = ARIMAForecaster("rainfall_forecaster", message_bus, forecaster_config)
    agent_components.append(rainfall_forecaster)

    # 扰动智能体
    inflow_agent_cfg = {**agent_cfgs['inflow_source_agent']['config'], 'topic': topics['upstream_inflow_data']}
    inflow_source_agent = RainfallAgent("inflow_source_agent", message_bus, inflow_agent_cfg)
    agent_components.append(inflow_source_agent)

    rain_agent_cfg = {**agent_cfgs['rainfall_disturbance_agent']['config'], 'topic': topics['upstream_inflow_data']}
    rainfall_agent = RainfallAgent("rainfall_agent", message_bus, rain_agent_cfg)
    agent_components.append(rainfall_agent)

    water_use_cfg = {**agent_cfgs['water_use_agent']['config'], 'topic': topics['upstream_inflow_data']}
    water_use_agent = WaterUseAgent("water_use_agent", message_bus, water_use_cfg)
    agent_components.append(water_use_agent)

    # IO 智能体
    io_agent = PhysicalIOAgent("io_agent_1", message_bus, {}, {'gate_actuator': {'obj': control_gate, 'target_attr': 'target_opening', 'topic': topics['gate_action'], 'control_key': 'control_signal'}})
    agent_components.append(io_agent)
    print("智能体组件配置完成。")

    # --- 搭建并运行仿真 ---
    harness = SimulationHarness(config={'duration': sim_cfg['duration'], 'dt': sim_cfg['dt']})
    for component in physical_components:
        harness.add_component(component)
    for agent in agent_components:
        harness.add_agent(agent)
    
    harness.add_connection("upstream_canal", "control_gate")
    harness.add_connection("control_gate", "downstream_canal")

    print("\n--- 开始运行仿真 ---")
    harness.build()
    harness.run_mas_simulation()
    print("--- 仿真结束 ---")
    return harness

### 4. 定义结果绘图函数

此函数用于处理仿真结果并将其可视化。

In [ ]:
def plot_results(harness, cfg):
    print("\n--- 正在绘制仿真结果 ---")
    history = harness.history
    if not history:
        print("没有历史数据可以绘制。")
        return

    flat_history = []
    for step in history:
        row = {'time': step['time']}
        for component_name, states in step.items():
            if component_name == 'time' or not isinstance(states, dict): continue
            for state_key, value in states.items():
                row[f"{component_name}.{state_key}"] = value
        flat_history.append(row)
    results = pd.DataFrame(flat_history).set_index('time')

    fig, axs = plt.subplots(3, 1, figsize=(14, 16), sharex=True)
    fig.suptitle('运河分流仿真结果', fontsize=16)

    # 图1: 上游水位与PID设定点
    axs[0].plot(results.index, results['upstream_canal.water_level'], label='上游水位 (m)', color='b')
    disp_rules = cfg['agents']['central_dispatcher']['rules']
    axs[0].axhline(y=disp_rules['normal_setpoint'], color='grey', linestyle=':', label=f"正常设定点 ({disp_rules['normal_setpoint']}m)")
    axs[0].axhline(y=disp_rules['emergency_setpoint'], color='red', linestyle=':', label=f"紧急设定点 ({disp_rules['emergency_setpoint']}m)")
    rain_start = cfg['agents']['rainfall_disturbance_agent']['config']['start_time']
    rain_end = rain_start + cfg['agents']['rainfall_disturbance_agent']['config']['duration']
    axs[0].axvline(x=rain_start, color='k', linestyle='--', label='扰动开始')
    axs[0].axvline(x=rain_end, color='k', linestyle='--')
    axs[0].set_ylabel('水位 (m)')
    axs[0].legend()
    axs[0].grid(True)
    axs[0].set_title('上游运河水位')

    # 图2: 水闸开度
    axs[1].plot(results.index, results['control_gate.opening'], label='水闸开度 (m)', color='g')
    axs[1].fill_between(results.index, 0, results['control_gate.opening'], color='g', alpha=0.2)
    axs[1].set_ylabel('开度 (m)')
    axs[1].legend()
    axs[1].grid(True)
    axs[1].set_title('控制水闸开度')

    # 图3: 流量动态
    axs[2].plot(results.index, results['upstream_canal.outflow'], label='上游运河出口流量 (m³/s)', color='m')
    # 重建净来水用于绘图
    base_inflow = cfg['simulation_settings']['normal_inflow']
    rain_inflow = cfg['agents']['rainfall_disturbance_agent']['config']['inflow_rate']
    water_use = cfg['agents']['water_use_agent']['config']['demand_rate']
    inflow = [base_inflow if t < rain_start else (base_inflow + rain_inflow) if t < rain_end else base_inflow for t in results.index]
    net_inflow = [val - water_use for val in inflow]
    axs[2].plot(results.index, net_inflow, label='净来水 (m³/s)', color='c')
    axs[2].set_xlabel('时间 (s)')
    axs[2].set_ylabel('流量 (m³/s)')
    axs[2].legend()
    axs[2].grid(True)
    axs[2].set_title('上游运河流量动态')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


### 5. 运行仿真并显示结果

In [ ]:
simulation_harness = run_simulation_from_config(config)
if simulation_harness:
    plot_results(simulation_harness, config)